# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 28 2022 9:23AM,249588,10,8703740-SD,Eywa Pharma Inc.,Released
1,Oct 28 2022 9:18AM,249587,16,8700664,Sartorius Lab Products and Service,Released
2,Oct 28 2022 9:18AM,249587,16,8700668,Sartorius Lab Products and Service,Released
3,Oct 28 2022 9:18AM,249587,16,8700671,Sartorius Lab Products and Service,Released
4,Oct 28 2022 9:18AM,249587,16,8700674,Sartorius Lab Products and Service,Released
5,Oct 28 2022 9:18AM,249587,16,8700678,Sartorius Lab Products and Service,Released
6,Oct 28 2022 9:18AM,249586,16,8700130,Sartorius Bioprocess Solutions,Released
7,Oct 28 2022 8:24AM,249577,16,EMERSA-8703231,"Emersa Waterbox, LLC",Released
8,Oct 27 2022 4:19PM,249572,15,PNC391151A,"Person & Covey, Inc.",Released
9,Oct 27 2022 4:19PM,249572,15,PNC391527A,"Person & Covey, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
13,249572,Released,28
14,249577,Released,1
15,249586,Released,1
16,249587,Released,5
17,249588,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
249572,NaN,28.0
249577,NaN,1.0
249586,NaN,1.0
249587,NaN,5.0
249588,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
249494,1.0,0.0
249519,0.0,1.0
249520,0.0,1.0
249521,0.0,1.0
249523,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
249494,1,0
249519,0,1
249520,0,1
249521,0,1
249523,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,249494,1,0
1,249519,0,1
2,249520,0,1
3,249521,0,1
4,249523,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,249494,1,
1,249519,,1
2,249520,,1
3,249521,,1
4,249523,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 28 2022 9:23AM,249588,10,Eywa Pharma Inc.
1,Oct 28 2022 9:18AM,249587,16,Sartorius Lab Products and Service
6,Oct 28 2022 9:18AM,249586,16,Sartorius Bioprocess Solutions
7,Oct 28 2022 8:24AM,249577,16,"Emersa Waterbox, LLC"
8,Oct 27 2022 4:19PM,249572,15,"Person & Covey, Inc."
36,Oct 27 2022 4:07PM,249570,16,Sartorius Stedim Filters Inc.
37,Oct 27 2022 3:16PM,249566,21,"NBTY Global, Inc."
38,Oct 27 2022 2:44PM,249555,21,"NBTY Global, Inc."
39,Oct 27 2022 2:39PM,249554,21,"NBTY Global, Inc."
40,Oct 27 2022 1:48PM,249544,15,"Person & Covey, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Oct 28 2022 9:23AM,249588,10,Eywa Pharma Inc.,,1
1,Oct 28 2022 9:18AM,249587,16,Sartorius Lab Products and Service,,5
2,Oct 28 2022 9:18AM,249586,16,Sartorius Bioprocess Solutions,,1
3,Oct 28 2022 8:24AM,249577,16,"Emersa Waterbox, LLC",,1
4,Oct 27 2022 4:19PM,249572,15,"Person & Covey, Inc.",,28
5,Oct 27 2022 4:07PM,249570,16,Sartorius Stedim Filters Inc.,1,
6,Oct 27 2022 3:16PM,249566,21,"NBTY Global, Inc.",,1
7,Oct 27 2022 2:44PM,249555,21,"NBTY Global, Inc.",1,
8,Oct 27 2022 2:39PM,249554,21,"NBTY Global, Inc.",1,
9,Oct 27 2022 1:48PM,249544,15,"Person & Covey, Inc.",,36


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 28 2022 9:23AM,249588,10,Eywa Pharma Inc.,1,
1,Oct 28 2022 9:18AM,249587,16,Sartorius Lab Products and Service,5,
2,Oct 28 2022 9:18AM,249586,16,Sartorius Bioprocess Solutions,1,
3,Oct 28 2022 8:24AM,249577,16,"Emersa Waterbox, LLC",1,
4,Oct 27 2022 4:19PM,249572,15,"Person & Covey, Inc.",28,
5,Oct 27 2022 4:07PM,249570,16,Sartorius Stedim Filters Inc.,,1
6,Oct 27 2022 3:16PM,249566,21,"NBTY Global, Inc.",1,
7,Oct 27 2022 2:44PM,249555,21,"NBTY Global, Inc.",,1
8,Oct 27 2022 2:39PM,249554,21,"NBTY Global, Inc.",,1
9,Oct 27 2022 1:48PM,249544,15,"Person & Covey, Inc.",36,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 28 2022 9:23AM,249588,10,Eywa Pharma Inc.,1,
1,Oct 28 2022 9:18AM,249587,16,Sartorius Lab Products and Service,5,
2,Oct 28 2022 9:18AM,249586,16,Sartorius Bioprocess Solutions,1,
3,Oct 28 2022 8:24AM,249577,16,"Emersa Waterbox, LLC",1,
4,Oct 27 2022 4:19PM,249572,15,"Person & Covey, Inc.",28,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Oct 28 2022 9:23AM,249588,10,Eywa Pharma Inc.,1.0,NaN
1,Oct 28 2022 9:18AM,249587,16,Sartorius Lab Products and Service,5.0,NaN
2,Oct 28 2022 9:18AM,249586,16,Sartorius Bioprocess Solutions,1.0,NaN
3,Oct 28 2022 8:24AM,249577,16,"Emersa Waterbox, LLC",1.0,NaN
4,Oct 27 2022 4:19PM,249572,15,"Person & Covey, Inc.",28.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 28 2022 9:23AM,249588,10,Eywa Pharma Inc.,1.0,0.0
1,Oct 28 2022 9:18AM,249587,16,Sartorius Lab Products and Service,5.0,0.0
2,Oct 28 2022 9:18AM,249586,16,Sartorius Bioprocess Solutions,1.0,0.0
3,Oct 28 2022 8:24AM,249577,16,"Emersa Waterbox, LLC",1.0,0.0
4,Oct 27 2022 4:19PM,249572,15,"Person & Covey, Inc.",28.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,249588,1.0,0.0
15,748646,103.0,0.0
16,1746880,10.0,1.0
20,249494,0.0,1.0
21,1497248,4.0,2.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,249588,1.0,0.0
1,15,748646,103.0,0.0
2,16,1746880,10.0,1.0
3,20,249494,0.0,1.0
4,21,1497248,4.0,2.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,1.0,0.0
1,15,103.0,0.0
2,16,10.0,1.0
3,20,0.0,1.0
4,21,4.0,2.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,1.0
1,15,Released,103.0
2,16,Released,10.0
3,20,Released,0.0
4,21,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,20,21
Status,,,,,
Executing,0.0,0.0,1.0,1.0,2.0
Released,1.0,103.0,10.0,0.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,20,21
0,Executing,0.0,0.0,1.0,1.0,2.0
1,Released,1.0,103.0,10.0,0.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,20,21
0,Executing,0.0,0.0,1.0,1.0,2.0
1,Released,1.0,103.0,10.0,0.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()